# Cross-Architecture, Cross-Dataset Meta-Analysis of Walk Resolution Limit Theory

This notebook performs a comprehensive meta-analysis evaluating the **Walk Resolution Limit Theory** across 4 GNN architectures (model-free, MLP, GCN, GPS) and 4 datasets (ZINC, Peptides-func, Peptides-struct, Synthetic).

**5 Meta-Analyses performed:**
1. Architecture comparison via Spearman ρ with Fisher-z combination
2. SRI vs graph size confound analysis via cross-validated R²
3. Task-type moderation via Cohen's d
4. SRWE consistency scorecard with win rates
5. Scope-of-validity tiered assessment

**Key finding:** SRI theory holds mathematically (Tier A) but predictive power (Tier C) is not supported in neural architectures.

In [1]:
import subprocess, sys
def _pip(*a): subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', *a])

# loguru — NOT on Colab, always install
_pip('loguru==0.7.3')

# Core packages (pre-installed on Colab, install locally to match Colab env)
if 'google.colab' not in sys.modules:
    _pip('numpy==2.0.2', 'pandas==2.2.2', 'scikit-learn==1.6.1', 'scipy==1.16.3',
         'matplotlib==3.10.0', 'seaborn==0.13.2')


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import json
import math
import os
import sys
import warnings

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from loguru import logger

# Configure logging for notebook
logger.remove()
logger.add(sys.stdout, level="INFO", format="{time:HH:mm:ss}|{level:<7}|{message}")

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
GITHUB_DATA_URL = "https://raw.githubusercontent.com/AMGrobelnik/ai-invention-ace67e-the-walk-resolution-limit-a-super-resolu/main/evaluation_iter4_cross_architect/demo/mini_demo_data.json"
import json, os

def load_data():
    try:
        import urllib.request
        with urllib.request.urlopen(GITHUB_DATA_URL) as response:
            return json.loads(response.read().decode())
    except Exception: pass
    if os.path.exists("mini_demo_data.json"):
        with open("mini_demo_data.json") as f: return json.load(f)
    raise FileNotFoundError("Could not load mini_demo_data.json")

In [4]:
data = load_data()
# Extract the 4 bundled data sources
gps_raw = data["gps_data"]
gcn_raw = data["gcn_data"]
model_free_raw = data["model_free_data"]
diagnostics_raw = data["diagnostics_data"]
print(f"Loaded {len(data)} data sources: {list(data.keys())}")

Loaded 4 data sources: ['gps_data', 'gcn_data', 'model_free_data', 'diagnostics_data']


## Configuration

Tunable parameters for the meta-analysis. These control bootstrap iterations, cross-validation folds, and random forest hyperparameters.

In [ ]:
# ── Tunable parameters ──
# Original values from full evaluation script

# Bootstrap iterations for Spearman CI estimation
N_BOOT = 1000

# Cross-validation folds for R² estimation
N_SPLITS = 5

# Random forest hyperparameters for MA2
N_ESTIMATORS = 50
MAX_DEPTH = 5

# Max examples per dataset (0 = all)
MAX_EXAMPLES = 0  # use all available in mini data

## Utility Functions

Statistical helper functions for Spearman correlation (with bootstrap CIs), Fisher z-transform combination, partial correlation, Cohen's d effect sizes, cross-validated R², and per-graph loss computation.

In [6]:
def safe_spearman(x: np.ndarray, y: np.ndarray) -> tuple:
    """Compute Spearman correlation with error handling."""
    mask = np.isfinite(x) & np.isfinite(y)
    x, y = x[mask], y[mask]
    if len(x) < 5:
        return 0.0, 1.0
    try:
        rho, p = stats.spearmanr(x, y)
        return float(rho) if np.isfinite(rho) else 0.0, float(p) if np.isfinite(p) else 1.0
    except Exception:
        return 0.0, 1.0


def bootstrap_spearman(x: np.ndarray, y: np.ndarray, n_boot: int = None,
                        seed: int = 42) -> dict:
    """Spearman correlation with bootstrap 95% CI."""
    if n_boot is None:
        n_boot = N_BOOT
    rho, p = safe_spearman(x, y)
    mask = np.isfinite(x) & np.isfinite(y)
    x, y = x[mask], y[mask]
    n = len(x)
    if n < 10:
        return {"rho": rho, "p": p, "ci_low": rho, "ci_high": rho, "n": n}
    rng = np.random.RandomState(seed)
    boot_rhos = []
    for _ in range(n_boot):
        idx = rng.randint(0, n, size=n)
        r, _ = safe_spearman(x[idx], y[idx])
        boot_rhos.append(r)
    boot_rhos = np.array(boot_rhos)
    ci_low, ci_high = np.nanpercentile(boot_rhos, [2.5, 97.5])
    return {"rho": rho, "p": p, "ci_low": float(ci_low), "ci_high": float(ci_high), "n": n}


def fisher_z_combine(rhos: list, ns: list) -> dict:
    """Combine correlations using Fisher z-transform with weighted average."""
    if not rhos:
        return {"mean_rho": 0.0, "ci_low": 0.0, "ci_high": 0.0}
    zs = [np.arctanh(min(max(r, -0.999), 0.999)) for r in rhos]
    ws = [max(n - 3, 1) for n in ns]
    total_w = sum(ws)
    z_avg = sum(z * w for z, w in zip(zs, ws)) / total_w
    se_z = 1.0 / math.sqrt(total_w) if total_w > 0 else 1.0
    z_lo, z_hi = z_avg - 1.96 * se_z, z_avg + 1.96 * se_z
    return {
        "mean_rho": float(np.tanh(z_avg)),
        "ci_low": float(np.tanh(z_lo)),
        "ci_high": float(np.tanh(z_hi)),
    }


def partial_spearman(x: np.ndarray, y: np.ndarray, z: np.ndarray) -> tuple:
    """Partial Spearman correlation of x and y controlling for z via rank residualization."""
    mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(z)
    x, y, z = x[mask], y[mask], z[mask]
    if len(x) < 10:
        return 0.0, 1.0
    rx = stats.rankdata(x)
    ry = stats.rankdata(y)
    rz = stats.rankdata(z)
    from sklearn.linear_model import LinearRegression as LR
    res_x = rx - LR().fit(rz.reshape(-1, 1), rx).predict(rz.reshape(-1, 1))
    res_y = ry - LR().fit(rz.reshape(-1, 1), ry).predict(rz.reshape(-1, 1))
    rho, p = stats.spearmanr(res_x, res_y)
    return (float(rho) if np.isfinite(rho) else 0.0,
            float(p) if np.isfinite(p) else 1.0)


def cohens_d(group1: np.ndarray, group2: np.ndarray) -> float:
    """Compute Cohen's d effect size."""
    n1, n2 = len(group1), len(group2)
    if n1 < 2 or n2 < 2:
        return 0.0
    m1, m2 = np.nanmean(group1), np.nanmean(group2)
    s1, s2 = np.nanstd(group1, ddof=1), np.nanstd(group2, ddof=1)
    pooled_std = math.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
    if pooled_std < 1e-12:
        return 0.0
    return float((m1 - m2) / pooled_std)


def cv_r2(X: np.ndarray, y: np.ndarray, model_class, n_splits: int = None,
           seed: int = 42, **model_kwargs) -> float:
    """Cross-validated R² score."""
    if n_splits is None:
        n_splits = N_SPLITS
    mask = np.isfinite(X).all(axis=1) & np.isfinite(y)
    X, y = X[mask], y[mask]
    if len(X) < n_splits * 2:
        return 0.0
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    scores = []
    for train_idx, test_idx in kf.split(X):
        try:
            model = model_class(**model_kwargs)
            model.fit(X[train_idx], y[train_idx])
            pred = model.predict(X[test_idx])
            ss_res = np.sum((y[test_idx] - pred) ** 2)
            ss_tot = np.sum((y[test_idx] - np.mean(y[test_idx])) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot > 1e-12 else 0.0
            scores.append(r2)
        except Exception:
            scores.append(0.0)
    return float(np.mean(scores))


def parse_pred_string(s: str) -> np.ndarray:
    """Parse prediction string like '[1.0, 2.0]' to numpy array."""
    try:
        s = s.strip()
        if s.startswith("["):
            vals = json.loads(s)
        else:
            vals = [float(s)]
        return np.array(vals, dtype=float)
    except Exception:
        return np.array([float("nan")])


def compute_per_graph_mae(pred_str: str, true_str: str) -> float:
    """Compute per-graph MAE from prediction and ground truth strings."""
    try:
        pred = parse_pred_string(pred_str)
        true_val = parse_pred_string(true_str)
        if len(pred) != len(true_val):
            return float("nan")
        return float(np.mean(np.abs(pred - true_val)))
    except Exception:
        return float("nan")


def compute_per_graph_bce(pred_str: str, true_str: str) -> float:
    """Compute per-graph binary cross-entropy from prediction and ground truth strings."""
    try:
        pred = parse_pred_string(pred_str)
        true_val = parse_pred_string(true_str)
        if len(pred) != len(true_val):
            return float("nan")
        pred_clipped = np.clip(pred, 1e-7, 1 - 1e-7)
        bce = -np.mean(true_val * np.log(pred_clipped) + (1 - true_val) * np.log(1 - pred_clipped))
        return float(bce) if np.isfinite(bce) else float("nan")
    except Exception:
        return float("nan")

print("Utility functions defined.")

Utility functions defined.


## Data Extraction

Extract per-graph data from GPS, GCN experiments and summary statistics from model-free and diagnostics experiments.

In [7]:
def extract_gps_per_graph(data: dict) -> pd.DataFrame:
    """Extract per-graph data from GPS experiment (exp_id1_it3)."""
    rows = []
    for ds in data["datasets"]:
        ds_name = ds["dataset"]
        for ex in ds["examples"]:
            rows.append({
                "dataset": ds_name,
                "architecture": "GPS",
                "sri": ex.get("metadata_sri_k20", float("nan")),
                "num_nodes": ex.get("metadata_num_nodes", float("nan")),
                "gap_rwse_lappe": ex.get("metadata_gap_rwse_lappe", float("nan")),
                "gap_srwe_lappe": ex.get("metadata_gap_srwe_lappe", float("nan")),
                "loss_rwse": ex.get("metadata_loss_rwse", float("nan")),
                "loss_lappe": ex.get("metadata_loss_lappe", float("nan")),
                "loss_srwe": ex.get("metadata_loss_srwe", float("nan")),
                "task_type": ex.get("metadata_task_type", "unknown"),
            })
    df = pd.DataFrame(rows)
    logger.info(f"GPS per-graph data: {len(df)} rows, datasets: {df['dataset'].unique().tolist()}")
    return df


def extract_gcn_per_graph(data: dict) -> pd.DataFrame:
    """Extract per-graph data from GCN experiment (exp_id2_it3)."""
    rows = []
    for ds in data["datasets"]:
        ds_name = ds["dataset"]
        for ex in ds["examples"]:
            sri = ex.get("metadata_sri_k20", float("nan"))
            true_str = ex.get("output", "")
            pred_rwse = ex.get("predict_rwse", "")
            pred_lappe = ex.get("predict_lappe", "")
            pred_srwe = ex.get("predict_srwe", "")

            if ds_name in ("ZINC-subset",):
                task_type = "regression"
                loss_rwse = compute_per_graph_mae(pred_rwse, true_str)
                loss_lappe = compute_per_graph_mae(pred_lappe, true_str)
                loss_srwe = compute_per_graph_mae(pred_srwe, true_str)
            elif ds_name in ("Peptides-struct",):
                task_type = "regression"
                loss_rwse = compute_per_graph_mae(pred_rwse, true_str)
                loss_lappe = compute_per_graph_mae(pred_lappe, true_str)
                loss_srwe = compute_per_graph_mae(pred_srwe, true_str)
            elif ds_name in ("Peptides-func",):
                task_type = "classification"
                loss_rwse = compute_per_graph_bce(pred_rwse, true_str)
                loss_lappe = compute_per_graph_bce(pred_lappe, true_str)
                loss_srwe = compute_per_graph_bce(pred_srwe, true_str)
            elif ds_name in ("Synthetic-aliased-pairs",):
                task_type = "regression"
                loss_rwse = compute_per_graph_mae(pred_rwse, true_str)
                loss_lappe = compute_per_graph_mae(pred_lappe, true_str)
                loss_srwe = compute_per_graph_mae(pred_srwe, true_str)
            else:
                task_type = "unknown"
                loss_rwse = loss_lappe = loss_srwe = float("nan")

            gap = loss_rwse - loss_lappe if np.isfinite(loss_rwse) and np.isfinite(loss_lappe) else float("nan")
            gap_srwe = loss_srwe - loss_lappe if np.isfinite(loss_srwe) and np.isfinite(loss_lappe) else float("nan")

            num_nodes = float("nan")
            try:
                inp = json.loads(ex.get("input", "{}"))
                if "num_nodes" in inp:
                    num_nodes = float(inp["num_nodes"])
                elif "edge_index" in inp:
                    edge_index = inp["edge_index"]
                    if isinstance(edge_index, list) and len(edge_index) == 2:
                        all_nodes = set(edge_index[0] + edge_index[1])
                        num_nodes = float(len(all_nodes))
            except Exception:
                pass

            rows.append({
                "dataset": ds_name,
                "architecture": "GCN",
                "sri": sri,
                "num_nodes": num_nodes,
                "gap_rwse_lappe": gap,
                "gap_srwe_lappe": gap_srwe,
                "loss_rwse": loss_rwse,
                "loss_lappe": loss_lappe,
                "loss_srwe": loss_srwe,
                "task_type": task_type,
            })
    df = pd.DataFrame(rows)
    logger.info(f"GCN per-graph data: {len(df)} rows, datasets: {df['dataset'].unique().tolist()}")
    return df


def extract_model_free_summary(data: dict) -> dict:
    """Extract summary-level correlations from model-free/MLP experiment."""
    meta = data.get("metadata", {})
    results = {}
    phase2 = meta.get("phases", {}).get("phase2_model_free_quality", {})
    for ds_name, ds_data in phase2.items():
        rho_info = ds_data.get("spearman_sri_vs_gap", {})
        results[f"model_free_{ds_name}"] = {
            "rho": rho_info.get("rho", 0.0),
            "p": rho_info.get("p", 1.0),
            "n": ds_data.get("n_valid", 0),
        }
    phase4 = meta.get("phases", {}).get("phase4_correlation", {})
    for ds_name, ds_data in phase4.items():
        primary = ds_data.get("primary", {}).get("sri_vs_gap", {})
        bootstrap = primary.get("bootstrap", {})
        results[f"MLP_{ds_name}"] = {
            "rho": primary.get("rho", 0.0),
            "p": primary.get("p", 1.0),
            "n": ds_data.get("n_test", 0),
            "ci_low": bootstrap.get("ci_low", 0.0),
            "ci_high": bootstrap.get("ci_high", 0.0),
        }
        size_ctrl = ds_data.get("size_controlled", [])
        results[f"MLP_{ds_name}_size_ctrl"] = size_ctrl
        conf = ds_data.get("confounder", {}).get("sri_vs_num_nodes", {})
        results[f"MLP_{ds_name}_sri_size_corr"] = conf.get("rho", 0.0)
    return results


def extract_diagnostics_summary(data: dict) -> dict:
    """Extract summary-level results from spectral diagnostics experiment."""
    meta = data.get("metadata", {})
    return {
        "vandermonde": meta.get("analysis_4_reconstruction", {}),
        "vandermonde_conditioning": meta.get("analysis_4_vandermonde_conditioning", {}),
        "sri_distributions": meta.get("analysis_2_sri_distributions", {}),
        "summary": meta.get("summary", {}),
    }


# Extract all data
gps_df = extract_gps_per_graph(gps_raw)
gcn_df = extract_gcn_per_graph(gcn_raw)
model_free_summary = extract_model_free_summary(model_free_raw)
diagnostics_summary = extract_diagnostics_summary(diagnostics_raw)

# Apply MAX_EXAMPLES limit if set
if MAX_EXAMPLES > 0:
    gps_dfs = []
    for ds in gps_df["dataset"].unique():
        gps_dfs.append(gps_df[gps_df["dataset"] == ds].head(MAX_EXAMPLES))
    gps_df = pd.concat(gps_dfs, ignore_index=True)
    gcn_dfs = []
    for ds in gcn_df["dataset"].unique():
        gcn_dfs.append(gcn_df[gcn_df["dataset"] == ds].head(MAX_EXAMPLES))
    gcn_df = pd.concat(gcn_dfs, ignore_index=True)

print(f"GPS: {len(gps_df)} examples, GCN: {len(gcn_df)} examples")
print(f"GPS datasets: {gps_df['dataset'].unique().tolist()}")
print(f"GCN datasets: {gcn_df['dataset'].unique().tolist()}")

19:17:33|INFO   |GPS per-graph data: 95 rows, datasets: ['ZINC-subset', 'Peptides-func', 'Peptides-struct', 'Synthetic-aliased-pairs']


19:17:33|INFO   |GCN per-graph data: 100 rows, datasets: ['ZINC-subset', 'Peptides-func', 'Peptides-struct', 'Synthetic-aliased-pairs']


GPS: 95 examples, GCN: 100 examples
GPS datasets: ['ZINC-subset', 'Peptides-func', 'Peptides-struct', 'Synthetic-aliased-pairs']
GCN datasets: ['ZINC-subset', 'Peptides-func', 'Peptides-struct', 'Synthetic-aliased-pairs']


## Meta-Analysis 1: Architecture Comparison

Compare SRI-gap correlations across architectures using Spearman ρ with Fisher-z combination across datasets.

In [8]:
def meta_analysis_1_architecture_comparison(gps_df, gcn_df, model_free_summary):
    """Compare SRI-gap correlations across architectures."""
    logger.info("=== META-ANALYSIS 1: Architecture Comparison ===")

    datasets_of_interest = ["ZINC-subset", "Peptides-func", "Peptides-struct", "Synthetic-aliased-pairs"]
    architectures = ["model_free", "MLP", "GCN", "GPS"]
    results = {"per_dataset_architecture": {}, "heatmap_data": {}}

    for ds in datasets_of_interest:
        results["per_dataset_architecture"][ds] = {}
        key = f"model_free_{ds}"
        if key in model_free_summary:
            info = model_free_summary[key]
            results["per_dataset_architecture"][ds]["model_free"] = {
                "rho": info["rho"], "p": info["p"], "n": info["n"],
                "ci_low": info["rho"] - 0.05, "ci_high": info["rho"] + 0.05,
            }
        key = f"MLP_{ds}"
        if key in model_free_summary:
            info = model_free_summary[key]
            results["per_dataset_architecture"][ds]["MLP"] = {
                "rho": info["rho"], "p": info["p"], "n": info["n"],
                "ci_low": info.get("ci_low", 0.0), "ci_high": info.get("ci_high", 0.0),
            }
        gcn_ds = gcn_df[gcn_df["dataset"] == ds]
        if len(gcn_ds) > 5:
            sri = gcn_ds["sri"].values
            gap = gcn_ds["gap_rwse_lappe"].values
            boot = bootstrap_spearman(sri, gap)
            results["per_dataset_architecture"][ds]["GCN"] = boot
        gps_ds = gps_df[gps_df["dataset"] == ds]
        if len(gps_ds) > 5:
            sri = gps_ds["sri"].values
            gap = gps_ds["gap_rwse_lappe"].values
            boot = bootstrap_spearman(sri, gap)
            results["per_dataset_architecture"][ds]["GPS"] = boot

    heatmap = np.full((len(datasets_of_interest), len(architectures)), np.nan)
    for i, ds in enumerate(datasets_of_interest):
        for j, arch in enumerate(architectures):
            if ds in results["per_dataset_architecture"]:
                if arch in results["per_dataset_architecture"][ds]:
                    heatmap[i, j] = results["per_dataset_architecture"][ds][arch]["rho"]
    results["heatmap_data"] = {"matrix": heatmap.tolist(), "rows": datasets_of_interest, "cols": architectures}

    for arch in architectures:
        rhos, ns = [], []
        for ds in datasets_of_interest:
            if ds in results["per_dataset_architecture"]:
                if arch in results["per_dataset_architecture"][ds]:
                    entry = results["per_dataset_architecture"][ds][arch]
                    rhos.append(entry["rho"])
                    ns.append(entry.get("n", 100))
        combined = fisher_z_combine(rhos, ns)
        results[f"fisher_z_{arch}"] = combined
        logger.info(f"  {arch}: Fisher-z mean rho = {combined['mean_rho']:.4f} "
                     f"[{combined['ci_low']:.4f}, {combined['ci_high']:.4f}]")

    all_rhos = [results[f"fisher_z_{a}"]["mean_rho"] for a in architectures if f"fisher_z_{a}" in results]
    results["overall_mean_rho"] = float(np.mean(all_rhos)) if all_rhos else 0.0
    logger.info(f"  Overall mean rho across architectures: {results['overall_mean_rho']:.4f}")
    return results

ma1 = meta_analysis_1_architecture_comparison(gps_df, gcn_df, model_free_summary)

19:17:33|INFO   |=== META-ANALYSIS 1: Architecture Comparison ===


19:17:33|INFO   |  model_free: Fisher-z mean rho = 0.4070 [0.3940, 0.4198]


19:17:33|INFO   |  MLP: Fisher-z mean rho = 0.0682 [0.0282, 0.1079]


19:17:33|INFO   |  GCN: Fisher-z mean rho = 0.0417 [-0.1657, 0.2455]


19:17:33|INFO   |  GPS: Fisher-z mean rho = -0.0797 [-0.2868, 0.1344]


19:17:33|INFO   |  Overall mean rho across architectures: 0.1093


## Meta-Analysis 2: SRI vs Size Head-to-Head

Compare SRI vs graph size as predictors of performance gap using cross-validated R² and partial correlations.

In [9]:
def meta_analysis_2_sri_vs_size(gps_df, gcn_df):
    """Head-to-head comparison of SRI vs graph size as predictors of performance gap."""
    logger.info("=== META-ANALYSIS 2: SRI vs Size Head-to-Head ===")
    results = {}
    datasets_of_interest = ["ZINC-subset", "Peptides-func", "Peptides-struct"]

    for arch_name, df in [("GPS", gps_df), ("GCN", gcn_df)]:
        results[arch_name] = {}
        for ds in datasets_of_interest:
            ds_df = df[(df["dataset"] == ds)].copy()
            mask = (np.isfinite(ds_df["sri"].values) &
                    np.isfinite(ds_df["num_nodes"].values) &
                    np.isfinite(ds_df["gap_rwse_lappe"].values))
            ds_df = ds_df[mask]
            if len(ds_df) < 20:
                logger.warning(f"  {arch_name}/{ds}: too few examples ({len(ds_df)}), skipping")
                continue

            sri = ds_df["sri"].values.astype(float)
            num_nodes = ds_df["num_nodes"].values.astype(float)
            gap = ds_df["gap_rwse_lappe"].values.astype(float)
            ds_result = {"n": len(ds_df)}

            X_sri = sri.reshape(-1, 1)
            X_size = num_nodes.reshape(-1, 1)
            X_both = np.column_stack([sri, num_nodes])
            lr_tmp = LinearRegression().fit(X_size, sri)
            sri_resid = sri - lr_tmp.predict(X_size)
            X_resid = sri_resid.reshape(-1, 1)

            for model_name, model_cls, model_kwargs in [
                ("LinearRegression", LinearRegression, {}),
                ("RandomForest", RandomForestRegressor,
                 {"n_estimators": N_ESTIMATORS, "max_depth": MAX_DEPTH, "random_state": 42, "n_jobs": 1}),
            ]:
                r2_sri = cv_r2(X_sri, gap, model_cls, **model_kwargs)
                r2_size = cv_r2(X_size, gap, model_cls, **model_kwargs)
                r2_both = cv_r2(X_both, gap, model_cls, **model_kwargs)
                r2_resid = cv_r2(X_resid, gap, model_cls, **model_kwargs)
                delta_r2 = r2_both - r2_size
                ds_result[model_name] = {
                    "r2_sri_alone": r2_sri, "r2_size_alone": r2_size,
                    "r2_sri_plus_size": r2_both, "r2_sri_residualized": r2_resid,
                    "delta_r2": delta_r2,
                }
                logger.info(f"  {arch_name}/{ds}/{model_name}: R²(SRI)={r2_sri:.4f}, "
                             f"R²(size)={r2_size:.4f}, ΔR²={delta_r2:.4f}")

            partial_rho, partial_p = partial_spearman(sri, gap, num_nodes)
            ds_result["partial_spearman"] = {"rho": partial_rho, "p": partial_p}
            sri_size_rho, sri_size_p = safe_spearman(sri, num_nodes)
            ds_result["sri_size_corr"] = {"rho": sri_size_rho, "p": sri_size_p}
            results[arch_name][ds] = ds_result

    all_delta_r2_lr, all_delta_r2_rf, all_partial_rho = [], [], []
    for arch in ["GPS", "GCN"]:
        for ds in datasets_of_interest:
            if ds in results.get(arch, {}):
                r = results[arch][ds]
                if "LinearRegression" in r: all_delta_r2_lr.append(r["LinearRegression"]["delta_r2"])
                if "RandomForest" in r: all_delta_r2_rf.append(r["RandomForest"]["delta_r2"])
                if "partial_spearman" in r: all_partial_rho.append(r["partial_spearman"]["rho"])

    results["aggregate"] = {
        "mean_delta_r2_lr": float(np.mean(all_delta_r2_lr)) if all_delta_r2_lr else 0.0,
        "mean_delta_r2_rf": float(np.mean(all_delta_r2_rf)) if all_delta_r2_rf else 0.0,
        "mean_partial_rho": float(np.mean(all_partial_rho)) if all_partial_rho else 0.0,
    }
    logger.info(f"  Aggregate: ΔR²(LR)={results['aggregate']['mean_delta_r2_lr']:.4f}, "
                 f"partial ρ={results['aggregate']['mean_partial_rho']:.4f}")
    return results

ma2 = meta_analysis_2_sri_vs_size(gps_df, gcn_df)

19:17:33|INFO   |=== META-ANALYSIS 2: SRI vs Size Head-to-Head ===


19:17:33|INFO   |  GPS/ZINC-subset/LinearRegression: R²(SRI)=-0.4572, R²(size)=-0.4137, ΔR²=-0.0372


19:17:33|INFO   |  GPS/ZINC-subset/RandomForest: R²(SRI)=-1.6624, R²(size)=-0.4921, ΔR²=-0.7666


19:17:33|INFO   |  GPS/Peptides-func/LinearRegression: R²(SRI)=-2.7134, R²(size)=-0.8669, ΔR²=-3.3664


19:17:33|INFO   |  GPS/Peptides-func/RandomForest: R²(SRI)=-0.6758, R²(size)=-0.7649, ΔR²=-0.1464


...

19:17:33|INFO   |  GCN/Peptides-func/LinearRegression: R²(SRI)=-1.2733, R²(size)=-0.7116, ΔR²=-0.2240


19:17:33|INFO   |  GCN/Peptides-func/RandomForest: R²(SRI)=-1.4403, R²(size)=-1.4579, ΔR²=0.0010


19:17:33|INFO   |  GCN/Peptides-struct/LinearRegression: R²(SRI)=-0.7187, R²(size)=-0.2040, ΔR²=-0.1421


19:17:33|INFO   |  GCN/Peptides-struct/RandomForest: R²(SRI)=-0.1900, R²(size)=-0.2967, ΔR²=0.1724


19:17:33|INFO   |  Aggregate: ΔR²(LR)=-0.8945, partial ρ=0.1967


## Meta-Analysis 3: Task-Type Analysis

Analyze whether walk resolution limit effects differ by task type (regression vs classification) using Cohen's d effect sizes.

In [10]:
def meta_analysis_3_task_type(gps_df, gcn_df, gps_summary, gcn_summary):
    """Analyze whether walk resolution limit effects differ by task type."""
    logger.info("=== META-ANALYSIS 3: Task-Type Analysis ===")
    results = {}
    regression_datasets = ["ZINC-subset", "Peptides-struct"]
    classification_datasets = ["Peptides-func"]

    for arch_name, summary_data in [("GPS", gps_summary), ("GCN", gcn_summary)]:
        arch_results = {}
        if arch_name == "GPS":
            summary = summary_data.get("metadata", {}).get("results_summary", {})
        else:
            summary = summary_data.get("metadata", {}).get("gnn_benchmark", {})

        for ds_name in ["ZINC-subset", "Peptides-func", "Peptides-struct"]:
            if arch_name == "GPS" and ds_name in summary:
                imp = summary[ds_name].get("srwe_improvement", {})
                arch_results[ds_name] = {
                    "gap_rwse_lappe": imp.get("mean_gap_rwse_lappe", float("nan")),
                    "gap_srwe_lappe": imp.get("mean_gap_srwe_lappe", float("nan")),
                    "gap_reduction": imp.get("gap_reduction_fraction", float("nan")),
                }
            elif arch_name == "GCN" and ds_name in summary:
                ds_info = summary[ds_name]
                res = ds_info.get("results", {})
                metric_type = ds_info.get("metric", "MAE")
                rwse_mean = res.get("rwse", {}).get("mean", float("nan"))
                lappe_mean = res.get("lappe", {}).get("mean", float("nan"))
                srwe_mean = res.get("srwe", {}).get("mean", float("nan"))
                if metric_type == "AP":
                    gap_rl = rwse_mean - lappe_mean
                    gap_sl = srwe_mean - lappe_mean
                else:
                    gap_rl = rwse_mean - lappe_mean
                    gap_sl = srwe_mean - lappe_mean
                gap_reduction = 0.0
                if abs(gap_rl) > 1e-10:
                    gap_reduction = 1.0 - gap_sl / gap_rl
                arch_results[ds_name] = {
                    "gap_rwse_lappe": gap_rl, "gap_srwe_lappe": gap_sl,
                    "gap_reduction": gap_reduction,
                    "rwse_mean": rwse_mean, "lappe_mean": lappe_mean, "srwe_mean": srwe_mean,
                }
        results[arch_name] = arch_results

    for arch_name, df in [("GPS", gps_df), ("GCN", gcn_df)]:
        regression_gaps = df[df["task_type"] == "regression"]["gap_rwse_lappe"].dropna().values
        classification_gaps = df[df["task_type"] == "classification"]["gap_rwse_lappe"].dropna().values
        if len(regression_gaps) > 5 and len(classification_gaps) > 5:
            d_gap = cohens_d(regression_gaps, classification_gaps)
            regression_srwe = df[df["task_type"] == "regression"]["gap_srwe_lappe"].dropna().values
            classification_srwe = df[df["task_type"] == "classification"]["gap_srwe_lappe"].dropna().values
            d_srwe = cohens_d(regression_srwe, classification_srwe) if (
                len(regression_srwe) > 5 and len(classification_srwe) > 5) else 0.0
            results[f"{arch_name}_cohens_d"] = {
                "rwse_lappe_gap": d_gap, "srwe_lappe_gap": d_srwe,
                "n_regression": len(regression_gaps), "n_classification": len(classification_gaps),
            }
            logger.info(f"  {arch_name} Cohen's d (RWSE-LapPE gap): {d_gap:.4f}")

    for arch_name, df in [("GPS", gps_df), ("GCN", gcn_df)]:
        for task_type in ["regression", "classification"]:
            tt_df = df[df["task_type"] == task_type]
            if len(tt_df) > 10:
                rho, p = safe_spearman(tt_df["sri"].values, tt_df["gap_rwse_lappe"].values)
                results[f"{arch_name}_{task_type}_sri_gap_corr"] = {"rho": rho, "p": p, "n": len(tt_df)}

    regression_rhos, classification_rhos = [], []
    for arch_name, df in [("GPS", gps_df), ("GCN", gcn_df)]:
        for ds in df["dataset"].unique():
            ds_df = df[df["dataset"] == ds]
            if len(ds_df) < 10: continue
            tt = ds_df["task_type"].iloc[0]
            rho, _ = safe_spearman(ds_df["sri"].values, ds_df["gap_rwse_lappe"].values)
            if tt == "regression": regression_rhos.append(rho)
            elif tt == "classification": classification_rhos.append(rho)

    if len(regression_rhos) >= 2 and len(classification_rhos) >= 1:
        try:
            u_stat, u_p = stats.mannwhitneyu(regression_rhos, classification_rhos, alternative="two-sided")
            results["mann_whitney_task_type"] = {"U": float(u_stat), "p": float(u_p),
                "regression_rhos": regression_rhos, "classification_rhos": classification_rhos}
        except Exception:
            results["mann_whitney_task_type"] = {"U": 0.0, "p": 1.0}
    else:
        results["mann_whitney_task_type"] = {"U": 0.0, "p": 1.0, "note": "insufficient data"}
    return results

ma3 = meta_analysis_3_task_type(gps_df, gcn_df, gps_raw, gcn_raw)

19:17:33|INFO   |=== META-ANALYSIS 3: Task-Type Analysis ===


19:17:33|INFO   |  GPS Cohen's d (RWSE-LapPE gap): 0.3563


19:17:33|INFO   |  GCN Cohen's d (RWSE-LapPE gap): 0.0675


## Meta-Analysis 4: SRWE Consistency Scorecard

SRWE win rates across all architecture-dataset conditions, with stratified analysis by SRI regime and task type.

In [11]:
def meta_analysis_4_srwe_scorecard(gps_df, gcn_df, gps_summary, gcn_summary):
    """SRWE consistency scorecard: win rates, gap reduction, stratified conditions."""
    logger.info("=== META-ANALYSIS 4: SRWE Consistency Scorecard ===")
    results = {"conditions": [], "wins_vs_rwse": 0, "wins_vs_lappe": 0, "total": 0}
    datasets = ["ZINC-subset", "Peptides-func", "Peptides-struct"]

    for arch_name, summary_data in [("GPS", gps_summary), ("GCN", gcn_summary)]:
        if arch_name == "GPS":
            summary = summary_data.get("metadata", {}).get("results_summary", {})
        else:
            summary = summary_data.get("metadata", {}).get("gnn_benchmark", {})

        for ds_name in datasets:
            if ds_name not in summary: continue
            if arch_name == "GPS":
                enc_metrics = summary[ds_name].get("per_encoding_metrics", {})
                rwse_mean = enc_metrics.get("rwse", {}).get("mean", float("nan"))
                lappe_mean = enc_metrics.get("lappe", {}).get("mean", float("nan"))
                srwe_mean = enc_metrics.get("srwe", {}).get("mean", float("nan"))
                imp = summary[ds_name].get("srwe_improvement", {})
                gap_reduction = imp.get("gap_reduction_fraction", 0.0)
                task_type = "regression" if ds_name in ("ZINC-subset", "Peptides-struct") else "classification"
                if task_type == "classification":
                    srwe_wins_rwse = srwe_mean > rwse_mean
                    srwe_wins_lappe = srwe_mean > lappe_mean
                else:
                    srwe_wins_rwse = srwe_mean < rwse_mean
                    srwe_wins_lappe = srwe_mean < lappe_mean
            else:
                res = summary[ds_name].get("results", {})
                metric_type = summary[ds_name].get("metric", "MAE")
                rwse_mean = res.get("rwse", {}).get("mean", float("nan"))
                lappe_mean = res.get("lappe", {}).get("mean", float("nan"))
                srwe_mean = res.get("srwe", {}).get("mean", float("nan"))
                task_type = summary[ds_name].get("task_type", "unknown")
                gap_rl = rwse_mean - lappe_mean
                gap_sl = srwe_mean - lappe_mean
                gap_reduction = (1.0 - gap_sl / gap_rl) if abs(gap_rl) > 1e-10 else 0.0
                if metric_type == "AP":
                    srwe_wins_rwse = srwe_mean > rwse_mean
                    srwe_wins_lappe = srwe_mean > lappe_mean
                else:
                    srwe_wins_rwse = srwe_mean < rwse_mean
                    srwe_wins_lappe = srwe_mean < lappe_mean

            sri_medians = {"ZINC-subset": 1.02, "Peptides-func": 0.019, "Peptides-struct": 0.019}
            sri_regime = "high" if sri_medians.get(ds_name, 0) > 1.0 else "low"
            condition = {
                "architecture": arch_name, "dataset": ds_name, "task_type": task_type,
                "sri_regime": sri_regime, "rwse_mean": rwse_mean, "lappe_mean": lappe_mean,
                "srwe_mean": srwe_mean, "gap_reduction_pct": gap_reduction * 100,
                "srwe_wins_vs_rwse": bool(srwe_wins_rwse), "srwe_wins_vs_lappe": bool(srwe_wins_lappe),
            }
            results["conditions"].append(condition)
            results["total"] += 1
            if srwe_wins_rwse: results["wins_vs_rwse"] += 1
            if srwe_wins_lappe: results["wins_vs_lappe"] += 1

    total = results["total"]
    results["win_rate_vs_rwse"] = results["wins_vs_rwse"] / total if total > 0 else 0.0
    results["win_rate_vs_lappe"] = results["wins_vs_lappe"] / total if total > 0 else 0.0

    low_sri_regression = [c for c in results["conditions"] if c["sri_regime"] == "low" and c["task_type"] == "regression"]
    low_sri_classification = [c for c in results["conditions"] if c["sri_regime"] == "low" and c["task_type"] == "classification"]
    high_sri = [c for c in results["conditions"] if c["sri_regime"] == "high"]

    for label, group in [("low_sri_regression", low_sri_regression),
                          ("low_sri_classification", low_sri_classification), ("high_sri", high_sri)]:
        if group:
            results[f"stratified_{label}"] = {
                "n": len(group),
                "mean_gap_reduction": float(np.mean([c["gap_reduction_pct"] for c in group])),
                "win_rate_vs_rwse": sum(1 for c in group if c["srwe_wins_vs_rwse"]) / len(group),
                "win_rate_vs_lappe": sum(1 for c in group if c["srwe_wins_vs_lappe"]) / len(group),
            }

    logger.info(f"  Overall win rate vs RWSE: {results['win_rate_vs_rwse']:.2f} ({results['wins_vs_rwse']}/{total})")
    logger.info(f"  Overall win rate vs LapPE: {results['win_rate_vs_lappe']:.2f} ({results['wins_vs_lappe']}/{total})")
    return results

ma4 = meta_analysis_4_srwe_scorecard(gps_df, gcn_df, gps_raw, gcn_raw)

19:17:33|INFO   |=== META-ANALYSIS 4: SRWE Consistency Scorecard ===


19:17:33|INFO   |  Overall win rate vs RWSE: 0.50 (3/6)


19:17:33|INFO   |  Overall win rate vs LapPE: 0.67 (4/6)


## Meta-Analysis 5: Scope of Validity

Formal 4-tier evidence classification for walk resolution limit theory: mathematical validity, SRI as aliasing classifier, downstream predictive power, and SRWE practical utility.

In [12]:
def meta_analysis_5_scope_of_validity(ma1, ma2, ma3, ma4, diagnostics_summary):
    """Formal 4-tier evidence classification for walk resolution limit theory."""
    logger.info("=== META-ANALYSIS 5: Scope of Validity Assessment ===")

    # Tier A: Mathematical validity
    vander = diagnostics_summary.get("vandermonde", {})
    vander_cond = diagnostics_summary.get("vandermonde_conditioning", {})
    tier_a = {"tier": "A_mathematical_validity",
              "description": "Vandermonde conditioning confirms theoretical resolution limit", "evidence": {}}
    for ds_name in ["ZINC-subset", "Peptides-func", "Synthetic-aliased-pairs"]:
        if ds_name in vander:
            tier_a["evidence"][ds_name] = {"cond_vs_error_rho": vander[ds_name].get("corr_cond_vs_error", {}).get("spearman_rho", 0.0)}
        if ds_name in vander_cond:
            tier_a["evidence"].setdefault(ds_name, {})
            tier_a["evidence"][ds_name]["growth_rate_vs_sri_rho"] = (
                vander_cond[ds_name].get("corr_growth_rate_vs_sri", {}).get("spearman_rho", 0.0))
    rho_vals = [v.get("cond_vs_error_rho", 0) for v in tier_a["evidence"].values() if "cond_vs_error_rho" in v]
    tier_a["support_level"] = "strong" if (rho_vals and np.mean(np.abs(rho_vals)) > 0.3) else "moderate"

    # Tier B: SRI as aliasing classifier
    sri_dist = diagnostics_summary.get("sri_distributions", {})
    tier_b = {"tier": "B_sri_aliasing_classifier",
              "description": "SRI effectively separates aliased from well-resolved graphs", "evidence": {}}
    for ds_name in ["ZINC-subset", "Peptides-func", "Peptides-struct"]:
        if ds_name in sri_dist:
            pct_below_1 = sri_dist[ds_name].get("sri_20", {}).get("pct_below_1", 0.0)
            tier_b["evidence"][ds_name] = {"pct_aliased_sri_below_1": pct_below_1}
    ks_tests = sri_dist.get("ks_tests", {})
    ks_significant = sum(1 for v in ks_tests.values() if v.get("sri_20", {}).get("p_value", 1.0) < 0.001)
    tier_b["ks_significant_pairs"] = ks_significant
    tier_b["support_level"] = "strong" if ks_significant >= 3 else "moderate"

    # Tier C: Downstream predictive power
    tier_c = {"tier": "C_downstream_predictive_power",
              "description": "SRI predicts GNN performance gaps with partial control for confounders", "evidence": {}}
    for arch in ["model_free", "MLP", "GCN", "GPS"]:
        key = f"fisher_z_{arch}"
        if key in ma1: tier_c["evidence"][arch] = ma1[key]
    tier_c["sri_vs_size"] = {
        "mean_delta_r2_lr": ma2.get("aggregate", {}).get("mean_delta_r2_lr", 0.0),
        "mean_delta_r2_rf": ma2.get("aggregate", {}).get("mean_delta_r2_rf", 0.0),
        "mean_partial_rho": ma2.get("aggregate", {}).get("mean_partial_rho", 0.0),
    }
    delta_r2_lr = tier_c["sri_vs_size"]["mean_delta_r2_lr"]
    partial_rho = tier_c["sri_vs_size"]["mean_partial_rho"]
    if abs(partial_rho) > 0.1 and delta_r2_lr > 0.01: tier_c["support_level"] = "moderate"
    elif abs(partial_rho) > 0.05 or delta_r2_lr > 0.0: tier_c["support_level"] = "weak"
    else: tier_c["support_level"] = "not_supported"

    # Tier D: SRWE practical utility
    tier_d = {"tier": "D_srwe_utility",
              "description": "SRWE provides practical improvement under specific conditions", "evidence": {}}
    tier_d["overall_win_rate_vs_rwse"] = ma4.get("win_rate_vs_rwse", 0.0)
    tier_d["overall_win_rate_vs_lappe"] = ma4.get("win_rate_vs_lappe", 0.0)
    for key in ["stratified_low_sri_regression", "stratified_low_sri_classification", "stratified_high_sri"]:
        if key in ma4: tier_d[key] = ma4[key]
    if "GPS_cohens_d" in ma3: tier_d["task_type_modulation"] = ma3["GPS_cohens_d"]
    win_rate = tier_d["overall_win_rate_vs_rwse"]
    if win_rate >= 0.7: tier_d["support_level"] = "strong"
    elif win_rate >= 0.5: tier_d["support_level"] = "moderate"
    else: tier_d["support_level"] = "weak"

    decision = {
        "recommendation": ("Use SRI to diagnose spectral aliasing risk. For regression tasks on "
            "low-SRI graphs (SRI < 1), SRWE provides measurable improvement over RWSE. "
            "For classification tasks, standard RWSE/LapPE may suffice."),
        "conditions_where_srwe_helps": "Low SRI + regression tasks",
        "conditions_where_srwe_neutral_or_hurts": "High SRI graphs, classification tasks",
    }
    results = {"tier_a": tier_a, "tier_b": tier_b, "tier_c": tier_c, "tier_d": tier_d, "decision": decision}
    for tier_key in ["tier_a", "tier_b", "tier_c", "tier_d"]:
        logger.info(f"  {tier_key}: {results[tier_key]['support_level']}")
    return results

ma5 = meta_analysis_5_scope_of_validity(ma1, ma2, ma3, ma4, diagnostics_summary)

19:17:33|INFO   |=== META-ANALYSIS 5: Scope of Validity Assessment ===


19:17:33|INFO   |  tier_a: strong


19:17:33|INFO   |  tier_b: strong


19:17:33|INFO   |  tier_c: weak


19:17:33|INFO   |  tier_d: moderate


## Results Summary & Visualization

Print key results in a readable table and generate publication-quality figures summarizing the meta-analysis findings.

In [13]:
# ── Summary Table ──
print("=" * 70)
print("SUMMARY OF KEY FINDINGS")
print("=" * 70)

# MA1: Architecture comparison
print("\n--- MA1: Architecture Comparison (Fisher-z combined ρ) ---")
for arch in ["model_free", "MLP", "GCN", "GPS"]:
    key = f"fisher_z_{arch}"
    if key in ma1:
        r = ma1[key]
        print(f"  {arch:12s}: ρ = {r['mean_rho']:+.4f}  [{r['ci_low']:+.4f}, {r['ci_high']:+.4f}]")
print(f"  {'Overall':12s}: ρ = {ma1.get('overall_mean_rho', 0):.4f}")

# MA2: SRI vs Size
print("\n--- MA2: SRI vs Size (Aggregated) ---")
agg = ma2.get("aggregate", {})
print(f"  Mean ΔR² (LR): {agg.get('mean_delta_r2_lr', 0):.4f}")
print(f"  Mean ΔR² (RF): {agg.get('mean_delta_r2_rf', 0):.4f}")
print(f"  Mean partial ρ: {agg.get('mean_partial_rho', 0):.4f}")

# MA3: Task type
print("\n--- MA3: Task-Type Analysis ---")
for arch in ["GPS", "GCN"]:
    key = f"{arch}_cohens_d"
    if key in ma3:
        d = ma3[key]
        print(f"  {arch} Cohen's d (RWSE-LapPE gap): {d['rwse_lappe_gap']:.4f}")
        print(f"  {arch} Cohen's d (SRWE-LapPE gap): {d['srwe_lappe_gap']:.4f}")

# MA4: SRWE Scorecard
print("\n--- MA4: SRWE Consistency Scorecard ---")
print(f"  Win rate vs RWSE: {ma4.get('win_rate_vs_rwse', 0):.2f} ({ma4.get('wins_vs_rwse', 0)}/{ma4.get('total', 0)})")
print(f"  Win rate vs LapPE: {ma4.get('win_rate_vs_lappe', 0):.2f} ({ma4.get('wins_vs_lappe', 0)}/{ma4.get('total', 0)})")
for cond in ma4.get("conditions", []):
    print(f"    {cond['architecture']}/{cond['dataset']}: SRWE={cond['srwe_mean']:.4f}, "
          f"gap_reduction={cond['gap_reduction_pct']:.1f}%")

# MA5: Scope of Validity
print("\n--- MA5: Scope of Validity ---")
for tier_key in ["tier_a", "tier_b", "tier_c", "tier_d"]:
    tier = ma5[tier_key]
    print(f"  {tier['tier']}: {tier['support_level'].upper()}")
print(f"\n  Recommendation: {ma5['decision']['recommendation']}")
print("=" * 70)

# ── Figures ──
plt.rcParams.update({"font.size": 11, "figure.dpi": 150})

# Figure 1: Correlation Heatmap
hm = ma1.get("heatmap_data", {})
matrix = np.array(hm.get("matrix", [[0]]))
rows = hm.get("rows", [""])
cols = hm.get("cols", [""])

fig, ax = plt.subplots(figsize=(8, 5))
annot = np.empty_like(matrix, dtype=object)
for i in range(matrix.shape[0]):
    for j in range(matrix.shape[1]):
        annot[i, j] = "N/A" if np.isnan(matrix[i, j]) else f"{matrix[i, j]:.3f}"
mask = np.isnan(matrix)
matrix_filled = np.nan_to_num(matrix, nan=0.0)
sns.heatmap(matrix_filled, annot=annot, fmt="", cmap="RdBu_r", center=0,
            xticklabels=cols, yticklabels=rows, mask=mask,
            vmin=-0.5, vmax=0.5, ax=ax, cbar_kws={"label": "Spearman ρ(SRI, gap)"})
ax.set_title("MA1: SRI-Gap Correlation Across Architectures", fontsize=13, pad=12)
ax.set_xlabel("Architecture")
ax.set_ylabel("Dataset")
plt.tight_layout()
plt.show()

# Figure 2: SRWE Gap Reduction Bar Chart
conditions = ma4.get("conditions", [])
if conditions:
    fig, ax = plt.subplots(figsize=(10, 5))
    labels = [f"{c['architecture']}\n{c['dataset']}" for c in conditions]
    gap_reductions = [c["gap_reduction_pct"] for c in conditions]
    colors = ["#2196F3" if c["task_type"] == "regression" else "#FF9800" for c in conditions]
    bars = ax.bar(range(len(labels)), gap_reductions, color=colors, edgecolor="black", linewidth=0.5)
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, fontsize=8, rotation=45, ha="right")
    ax.set_ylabel("SRWE Gap Reduction (%)")
    ax.set_title("MA4: SRWE Gap Reduction Across Conditions", fontsize=13, pad=12)
    ax.axhline(y=0, color="black", linewidth=0.8, linestyle="-")
    reg_patch = mpatches.Patch(color="#2196F3", label="Regression")
    cls_patch = mpatches.Patch(color="#FF9800", label="Classification")
    ax.legend(handles=[reg_patch, cls_patch], loc="upper right")
    plt.tight_layout()
    plt.show()

# Figure 3: Scope of Validity Tier Summary
fig, ax = plt.subplots(figsize=(8, 4))
tier_names = ["Tier A:\nMathematical", "Tier B:\nSRI Classifier", "Tier C:\nPredictive Power", "Tier D:\nSRWE Utility"]
support_map = {"strong": 3, "moderate": 2, "weak": 1, "not_supported": 0}
tier_values = [support_map.get(ma5[f"tier_{c}"]["support_level"], 0) for c in "abcd"]
color_map = {3: "#4CAF50", 2: "#FF9800", 1: "#f44336", 0: "#9E9E9E"}
bar_colors = [color_map[v] for v in tier_values]
ax.barh(range(len(tier_names)), tier_values, color=bar_colors, edgecolor="black", linewidth=0.5)
ax.set_yticks(range(len(tier_names)))
ax.set_yticklabels(tier_names)
ax.set_xlabel("Support Level")
ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(["Not Supported", "Weak", "Moderate", "Strong"])
ax.set_title("MA5: Scope of Validity - Evidence Tiers", fontsize=13, pad=12)
plt.tight_layout()
plt.show()

print("\nAll meta-analyses complete.")

SUMMARY OF KEY FINDINGS

--- MA1: Architecture Comparison (Fisher-z combined ρ) ---
  model_free  : ρ = +0.4070  [+0.3940, +0.4198]
  MLP         : ρ = +0.0682  [+0.0282, +0.1079]
  GCN         : ρ = +0.0417  [-0.1657, +0.2455]
  GPS         : ρ = -0.0797  [-0.2868, +0.1344]
  Overall     : ρ = 0.1093

--- MA2: SRI vs Size (Aggregated) ---
  Mean ΔR² (LR): -0.8945
  Mean ΔR² (RF): -0.0835
  Mean partial ρ: 0.1967

--- MA3: Task-Type Analysis ---
  GPS Cohen's d (RWSE-LapPE gap): 0.3563
  GPS Cohen's d (SRWE-LapPE gap): 0.3162
  GCN Cohen's d (RWSE-LapPE gap): 0.0675
  GCN Cohen's d (SRWE-LapPE gap): 0.3428

--- MA4: SRWE Consistency Scorecard ---
  Win rate vs RWSE: 0.50 (3/6)
  Win rate vs LapPE: 0.67 (4/6)
    GPS/ZINC-subset: SRWE=0.2333, gap_reduction=34.5%
    GPS/Peptides-func: SRWE=0.2758, gap_reduction=-276.8%
    GPS/Peptides-struct: SRWE=17.5167, gap_reduction=57.7%
    GCN/ZINC-subset: SRWE=0.4367, gap_reduction=39.1%
    GCN/Peptides-func: SRWE=0.4198, gap_reduction=41.5%
 


All meta-analyses complete.
